In [ ]:
# =========================================================================
# Spectral Gap Analysis for Random 3-Regular Graphs
# =========================================================================
# Research Goal: Analyze the minimum energy gap (Δ_min) of the Adiabatic 
# Quantum Computing (AQC) Hamiltonian H(s) for Max-Cut on 3-regular graphs.
# This data will be used to study the connection between Δ_min and QAOA 
# performance, where AQC runtime scales as T ∝ 1/(Δ_min)^2.
# =========================================================================

import numpy as np
import networkx as nx
from scipy.linalg import eigh
import pandas as pd
import time
from typing import List, Tuple

print("=" * 70)
print("  AQC SPECTRAL GAP ANALYSIS FOR RANDOM 3-REGULAR GRAPHS")
print("=" * 70)

--- Starting AQC Spectral Gap Analysis ---
N_QUBITS: 10
NUM_GRAPHS: 200
Building H_Initial (Mixer) matrix...


TypeError: eigh() got an unexpected keyword argument 'turbo'

## 1. Configuration Parameters

Set the computational parameters for the spectral gap analysis:
- **N_QUBITS**: Number of qubits/nodes (max 10 for computational efficiency)
- **NUM_GRAPHS**: Number of random graph instances to analyze
- **S_RESOLUTION**: Number of interpolation points in s ∈ [0,1]


In [ ]:
# Configuration Settings
N_QUBITS = 10          # Maximum number of qubits (nodes in the graph)
NUM_GRAPHS = 200       # Number of random 3-regular graph instances
S_RESOLUTION = 200     # Number of points to sample along s ∈ [0, 1]

# Output filename
OUTPUT_FILENAME = f'Delta_min_3_regular_N{N_QUBITS}_{NUM_GRAPHS}graphs.csv'

# Validate configuration
if N_QUBITS % 2 != 0:
    print(f"⚠️  WARNING: N={N_QUBITS} is odd. For 3-regular graphs, N should be even.")
    
print(f"✓ Configuration loaded:")
print(f"  - N_QUBITS: {N_QUBITS}")
print(f"  - NUM_GRAPHS: {NUM_GRAPHS}")
print(f"  - S_RESOLUTION: {S_RESOLUTION}")
print(f"  - Hilbert space dimension: 2^{N_QUBITS} = {2**N_QUBITS}")
print(f"  - Output file: {OUTPUT_FILENAME}")


## 2. Pauli Matrices

Define the single-qubit Pauli operators that will be used to construct the many-body Hamiltonians.


In [ ]:
# Define single-qubit Pauli matrices
SIGMA_X = np.array([[0, 1], 
                     [1, 0]], dtype=complex)

SIGMA_Z = np.array([[1, 0], 
                     [0, -1]], dtype=complex)

IDENTITY = np.eye(2, dtype=complex)

print("✓ Pauli matrices defined:")
print(f"  σ_x:\n{SIGMA_X}")
print(f"  σ_z:\n{SIGMA_Z}")


## 3. Hamiltonian Construction Functions

Build the AQC Hamiltonian components:
- **H_initial**: The transverse field (mixer) Hamiltonian = -∑ᵢ X̂ᵢ
- **H_problem**: The Max-Cut (cost) Hamiltonian = ∑₍ᵢ,ⱼ₎∈E ẐᵢẐⱼ
- **H(s)**: The interpolated Hamiltonian = (1-s)H_initial + s·H_problem
